In [1]:
import pandas as pd
import requests
from PythonScripts.keys import KEY_TWO
import PythonScripts.data_clean as dc

In [2]:
# API Call to get list of LEGO themes and convert to DataFrame
themes = requests.get(f'https://brickset.com/api/v3.asmx/getThemes?apiKey={KEY_TWO}') 
data = themes.json()
theme_df = pd.json_normalize(data, 'themes')

print('Shape: ',theme_df.shape)
theme_df.head()

Shape:  (153, 5)


,theme,setCount,subthemeCount,yearFrom,yearTo
0,4 Juniors,24,5,2003,2004
1,Action Wheelers,9,0,2000,2001
2,Advanced models,35,12,2000,2012
3,Adventurers,72,4,1998,2003
4,Agents,13,0,2008,2009


In [3]:
# Drop themes older than 1999, themes with less than 50 sets, sets that aren't currently in production, 
#minifig theme, or miscellaneous theme
print('Starting theme_df shape: ',theme_df.shape)

mask = theme_df[(theme_df['yearFrom'] < 1999) | (theme_df['setCount'] < 50) | (theme_df['yearTo'] < 2022) |
         (theme_df['theme'] == 'Collectable Minifigures') | (theme_df['theme'] == 'Miscellaneous')].index
theme_df.drop(mask, inplace=True)

print('Resulting theme_df shape: ',theme_df.shape)

Starting theme_df shape:  (153, 5)
Resulting theme_df shape:  (17, 5)


In [4]:
# Generate sample theme list to use in 2nd API call. Convert list to string for API parameters.
theme_list = []
for item in theme_df['theme'].sample(3):
    theme_list.append(item)
param_string = ", ".join(theme_list)
print('Theme List: ',param_string)

Theme List:  Minecraft, Super Mario, Star Wars


In [5]:
# 2nd API call to get a full set list for themes in the theme list generated by first API call. 
# Convert to a Dataframe.
parameters = {'theme' : f'{param_string}', 'pageSize' : 2500}
set_list = requests.get(f"https://brickset.com/api/v3.asmx/getSets?apiKey={KEY_TWO}&userHash=&params={parameters}")
set_data = set_list.json()
set_df = pd.json_normalize(set_data,'sets')
print('set_df shape: ',set_df.shape)

set_df shape:  (1066, 44)


In [6]:
# Drop columns using helper function
dc.drop_columns(set_df)

print('set_df shape: ',set_df.shape)
set_df.head()

set_df shape:  (1066, 8)


,setID,number,name,year,theme,subtheme,pieces,rating
0,1394,3219,Mini TIE Fighter,2002,Star Wars,Mini Building Set,12.0,3.5
1,1469,3340,"Star Wars #1 - Emperor Palpatine, Darth Maul a...",2000,Star Wars,Minifig Pack,32.0,4.0
2,1470,3341,"Star Wars #2 - Luke Skywalker, Han Solo and Bo...",2000,Star Wars,Minifig Pack,22.0,4.2
3,1471,3342,Star Wars #3 - Chewbacca and 2 Biker Scouts,2000,Star Wars,Minifig Pack,22.0,4.3
4,1472,3343,Star Wars #4 - Battle Droid Commander and 2 Ba...,2000,Star Wars,Minifig Pack,30.0,3.9


In [7]:
# Drop rows where there is no rating for the set.
mask_two = set_df[set_df['rating'] == 0].index
set_df.drop(mask_two, inplace=True)
print('set_df shape: ',set_df.shape)

set_df shape:  (838, 8)


In [8]:
# Drop any rows if they have a NaN value in the pieces column
pieces_null = set_df['pieces'].isnull().values.any()
if pieces_null == True:
    set_df.dropna(subset=['pieces'], inplace=True)
print('set_df shape: ',set_df.shape)
set_df.head()

set_df shape:  (823, 8)


,setID,number,name,year,theme,subtheme,pieces,rating
0,1394,3219,Mini TIE Fighter,2002,Star Wars,Mini Building Set,12.0,3.5
1,1469,3340,"Star Wars #1 - Emperor Palpatine, Darth Maul a...",2000,Star Wars,Minifig Pack,32.0,4.0
2,1470,3341,"Star Wars #2 - Luke Skywalker, Han Solo and Bo...",2000,Star Wars,Minifig Pack,22.0,4.2
3,1471,3342,Star Wars #3 - Chewbacca and 2 Biker Scouts,2000,Star Wars,Minifig Pack,22.0,4.3
4,1472,3343,Star Wars #4 - Battle Droid Commander and 2 Ba...,2000,Star Wars,Minifig Pack,30.0,3.9


In [9]:
#Convert pieces, minimum age range, and maximum age range from floats to ints.
set_df['pieces'] = set_df['pieces'].astype(pd.Int64Dtype())
set_df.head()

,setID,number,name,year,theme,subtheme,pieces,rating
0,1394,3219,Mini TIE Fighter,2002,Star Wars,Mini Building Set,12,3.5
1,1469,3340,"Star Wars #1 - Emperor Palpatine, Darth Maul a...",2000,Star Wars,Minifig Pack,32,4.0
2,1470,3341,"Star Wars #2 - Luke Skywalker, Han Solo and Bo...",2000,Star Wars,Minifig Pack,22,4.2
3,1471,3342,Star Wars #3 - Chewbacca and 2 Biker Scouts,2000,Star Wars,Minifig Pack,22,4.3
4,1472,3343,Star Wars #4 - Battle Droid Commander and 2 Ba...,2000,Star Wars,Minifig Pack,30,3.9


In [10]:
# Rename column labels
print(f'Starting column labels: {set_df.columns}')

rename_dict = {
               'setID' : 'Set ID',
               'number' : 'Set Number',
               'name' : 'Set Name',
               'year' : 'Release Year',
               'theme' : 'Theme',
               'subtheme' : 'Subtheme',
               'pieces' : 'Number of Pieces',
               'rating' : 'Brickset Rating',
               }
                

set_df.rename(columns=rename_dict, inplace=True)
print(f'\nResulting column labels: {set_df.columns}')
set_df.shape

Starting column labels: Index(['setID', 'number', 'name', 'year', 'theme', 'subtheme', 'pieces',
       'rating'],
      dtype='object')

Resulting column labels: Index(['Set ID', 'Set Number', 'Set Name', 'Release Year', 'Theme', 'Subtheme',
       'Number of Pieces', 'Brickset Rating'],
      dtype='object')


(823, 8)

In [11]:
# Make a new DataFrame with a sample of 100 sets from set_df
if len(set_df.index) > 50:
    set_df = set_df.sample(50)
else:
    print('Restart notebook kernel and try again for a better sample')
set_df.shape

(50, 8)

In [12]:
# Save sample DataFrame to .csv for visualization in Tableau
file_path = dc.csv_path('theme_sample_set_list.csv')
set_df.to_csv(file_path)